#### Case 2: Liquid hydrogen ($LH_2$) in a 2033 $\text{m}^3$ tank

This application corresponds to liquid hydrogen storage in the NASA's Space Launch System. It consists of a 2033 $m^3$ storage tank [NASA](https://www.energy.gov/sites/default/files/2021-10/new-lh2-sphere.pdf) in a tank with 8.4 m diameter and 40 m height. It will be assumed the following operation scenarios:

* Daily boil-off rate of 0.1%  
* Storage at atmospheric pressure with continuous removal of boil-off gas

For purposes of the example, it is assumed that the storage tank is passively insulated with perlite.

03/09/2024: Illustration of how to use SciPy to optimise

In [1]:
# Scientific computing
import numpy as np

# Visualisation
import matplotlib.pyplot as plt

# Import the storage tank Class
from cryoevap.storage_tanks import Tank

# Import Cryogen class
from cryoevap.cryogens import Cryogen

#### Setup tank and cryogen properties


In [9]:
# Cylindrical storage tank properties
Q_roof = 0 # Roof heat ingress / W
d_i = 8 # Internal diameter / m
d_o = 8.4   # External diameter / m
T_air = 293.15 # Temperature of the environment K

# Set overall heat transfer coefficient through the walls for liquid and vapour
U_L = 3.73e-1 # W/m2/K
U_V = 3.73e-1 # W/m2/K

# Specify heat transfer rate at the bottom
# This will represent the heat conduction from the piping system
Q_b = 100 # W, 

# Vertically orientated cylindrical tank volume
V_tank = 2033 #m^3
V_tank = 500 #m^3

# Initial liquid filling / Dimensionless
LF = 0.90 

# Specify tank operating pressure
P = 101325 # Pa

# Initialize mid-scale tank
mid_tank = Tank(d_i, d_o, V_tank, LF)
mid_tank.set_HeatTransProps(U_L, U_V, T_air, Q_b, Q_roof, eta_w= 0.5)

# Keep the tank roof insulated
mid_tank.U_roof = U_V

# Initialise cryogen
hydrogen = Cryogen(name = "hydrogen")
hydrogen.set_coolprops(P)

# Set cryogen
mid_tank.cryogen = hydrogen

Calculate initial evaporation rate and transient period

In [10]:
# Calculate initial evaporation rate
print("The initial evaporation rate of " + hydrogen.name + " is %.1f kg/h" % (mid_tank.b_l_dot * 3600))

# Estimate transient period duration
print("Transient period = %.3f s " % mid_tank.tau)

# Minimum number of hours to achieve steady state 
tau_h = (np.floor(mid_tank.tau / 3600) + 1)

# Print simulation time of the transient period for short-term storage
print("Simulation time: %.0i h" % tau_h )

# Calculate boil-off rate
BOR = (mid_tank.b_l_dot * 24 * 3600) / (mid_tank.V * mid_tank.LF * mid_tank.cryogen.rho_L)
print("BOR = %.3f %%" % (BOR * 100))

The initial evaporation rate of hydrogen is 204.4 kg/h
Transient period = 2346.619 s 
Simulation time: 1 h
BOR = 15.385 %


### Simulation setup and execution

### Define optimisation parameters

In [11]:
# Define vertical spacing
dz = 0.1

# Calculate number of nodes
n_z = 1 + int(np.round(mid_tank.l_V/dz, 0))

# Define dimensionless computational grid
mid_tank.z_grid = np.linspace(0, 1, n_z)

# Define evaporation time as the transient period
evap_time = 3600 * 72

# evap_time = 3600
# Time step to plot each vapour temperature profile
mid_tank.plot_interval = evap_time/6

# Time step to record data, relevant for plotting integrated quantities such as
# the vapour to liquid heat transfer rate, Q_VL
mid_tank.time_interval = 60

# mid_tank.evaporate(evap_time)

In [5]:
from scipy.optimize import Bounds, minimize

# Minimum and maximum practical ranges
# of initial liquid filling

# 5% represent ballast voyage
# 95% is the safety limit for potential liquid thermal expansion
bounds = Bounds([0.05], [0.95])

# Define objective function
def BOR_function(LF):

    # Update liquid filling
    mid_tank.LF = LF

    # Execute simulation
    mid_tank.evaporate(evap_time)

    # Calculate objective function
    f = 1 - mid_tank.data['V_L'][-1] / mid_tank.data['V_L'][0]
    
    print("LF = %.3f, f=%.3e" % (LF, f))
    return f    

# Initial liquid filling to optimise

x0 = 0.9 
res = minimize(BOR_function, x0, method='trust-constr', options={'verbose': 1}, bounds=bounds)

/tmp/ipykernel_6300/4018950756.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("LF = %.3f, f=%.3e" % (LF, f))


LF = 0.900, f=1.588e-02
LF = 0.900, f=1.588e-02
LF = 1.173, f=6.613e-04
LF = 1.173, f=1.211e-03
LF = 0.900, f=1.540e-02
LF = 0.900, f=1.588e-02
LF = 0.801, f=1.694e-02
LF = 0.801, f=1.694e-02
LF = 0.801, f=1.694e-02
LF = 0.801, f=1.694e-02
LF = 0.801, f=1.694e-02
LF = 0.801, f=1.694e-02
LF = 0.801, f=1.694e-02
LF = 0.801, f=1.694e-02
LF = 0.806, f=1.687e-02
LF = 0.806, f=1.687e-02
LF = 0.869, f=1.618e-02
LF = 0.869, f=1.618e-02
LF = 0.943, f=1.549e-02
LF = 0.943, f=1.549e-02
LF = 0.925, f=1.565e-02
LF = 0.925, f=1.565e-02
LF = 0.924, f=1.565e-02
LF = 0.924, f=1.565e-02
LF = 0.874, f=1.613e-02
LF = 0.874, f=1.613e-02
LF = 0.871, f=1.617e-02


KeyboardInterrupt: 

Global optimisation

In [12]:
from scipy.optimize import Bounds, differential_evolution

# Minimum and maximum practical ranges
# of initial liquid filling
# 5% represent ballast voyage
# 95% is the safety limit for potential liquid thermal expansion
bounds = [(0.05, 0.95)]  # Bounds in differential_evolution are specified as a list of tuples

# Define objective function
def BOR_function(LF):

    # Update liquid filling
    mid_tank.LF = LF[0]  # LF is now an array, take the first element

    # Execute simulation
    mid_tank.evaporate(evap_time)

    # Calculate objective function
    f = 1 - mid_tank.data['V_L'][-1] / mid_tank.data['V_L'][0]
    
    print("LF = %.3f, f=%.3e" % (LF[0], f))
    return f    

# Perform global optimization using differential evolution
result = differential_evolution(BOR_function, bounds, strategy='best1bin', maxiter=1000, popsize=15, tol=1e-6, mutation=(0.5, 1), recombination=0.7, disp=True)

# Results
print("Best solution found: LF =", result.x[0])
print("Function value:", result.fun)


LF = 0.891, f=4.137e-01
LF = 0.424, f=6.242e-01
LF = 0.645, f=4.925e-01
LF = 0.848, f=4.253e-01
LF = 0.783, f=4.427e-01
LF = 0.400, f=6.458e-01
LF = 0.151, f=1.184e+00
LF = 0.274, f=8.191e-01
LF = 0.734, f=4.592e-01
LF = 0.215, f=9.400e-01
LF = 0.292, f=7.867e-01
LF = 0.695, f=4.730e-01
LF = 0.476, f=5.838e-01
LF = 0.051, f=2.403e+00
LF = 0.550, f=5.402e-01
LF = 0.066, f=2.001e+00
LF = 0.648, f=4.924e-01
LF = 0.561, f=5.314e-01
LF = 0.164, f=1.120e+00
LF = 0.675, f=4.808e-01
LF = 0.808, f=4.359e-01
LF = 0.938, f=4.040e-01
LF = 0.437, f=6.120e-01
LF = 0.444, f=6.097e-01
LF = 0.394, f=6.537e-01


### Visualisation of results

#### Vapour temperature

In [ ]:
mid_tank.plot_tv(t_unit='h')

Visualise liquid and vapour heat ingresses, $\dot{Q}_{\text{L}}$ and  $\dot{Q}_{\text{V}}$.

The plot also shows the vapour to liquid heat ingress, $\dot{Q}_{VL}$, and  the partition of the vapour heat ingress that is transferred to the interface by the wall directly, $\dot{Q}_{\text{V,w}}$

In [ ]:
# Specify y-axis units as W, and time units to hours
mid_tank.plot_Q(unit = 'W', t_unit = 'h')

#### Plot liquid volume

In [ ]:
mid_tank.plot_V_L(t_unit='h')

#### Plot evaporation rate, $\dot{B}_{\text{L}}$, and boil-off gas rate, $\dot{B}_{}$

In [ ]:
mid_tank.plot_BOG(unit='kg/h', t_unit='h')

Optional: CSV data export

If evaporation data is intended to be post-processed in another software, it can be exported readily with the help of the Pandas package.

In [ ]:
# Import pandas 
import pandas as pd

Plot average vapour and boil-off gas temperature

In [ ]:
mid_tank.plot_tv_BOG(t_unit='h')

#### References

U.S. Department of Energy. (2021, October). DOE/NASA Advances in Liquid Hydrogen Storage Workshop. Retrieved from [https://www.energy.gov/sites/default/files/2021-10/new-lh2-sphere.pdf]